In [2]:
import requests
import os
from dotenv import load_dotenv

In [ ]:
url = ""
load_dotenv()
API_KEY = os.getenv("API_KEY")
header = {"Authorization":API_KEY}
data = {"item":""}
incoming = requests.get(url,headers=header,data=data)
if incoming.status_code==200:
    print(incoming.json())

Translating text from German to English


In [1]:
from transformers import MarianMTModel, MarianTokenizer

c:\Users\muthu\Documents\bfh\BFH-offenburg-burda\bfh\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model_name = 'Helsinki-NLP/opus-mt-de-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


c:\Users\muthu\Documents\bfh\BFH-offenburg-burda\bfh\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\muthu\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-de-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\muthu\Documents\bfh\BFH-offenburg-burda\bfh\Lib\site-packages\transformers\model

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [3]:
def translate(text):
    # Tokenize the input text
    tokenized_text = tokenizer.prepare_seq2seq_batch([text], return_tensors='pt')
    
    # Perform the translation
    translation = model.generate(**tokenized_text)
    
    # Decode the translated text
    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)
    
    return translated_text

In [ ]:
src_texts = ["ei"]
inputs = tokenizer(src_texts, return_tensors="pt", padding=True, truncation=True)
translated = model.generate(**inputs)

# Decode and print result
translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(translated_texts[0])

Egg


In [12]:
inputs = ["Ei","öl","mehl"]
for src_texts in inputs:
    inputs = tokenizer(src_texts, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)

    # Decode and print result
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    print(translated_texts[0])


Egg
Oil
Flour


In [15]:
import json
import re
from typing import List, Dict, Any, Iterable

# --- basic helpers ---
def clean_spaces(s: str) -> str:
    """Collapse multiple spaces and trim."""
    return re.sub(r"\s+", " ", s).strip()

def join_ingredient_parts(item: dict) -> str:
    """Build a readable ingredient string including prefix/suffix if available."""
    prefix = item.get("prefix") or ""
    name = (item.get("ingredient") or {}).get("name", "") or ""
    suffix = item.get("suffix") or ""
    text = clean_spaces(f"{prefix}{name}")
    if suffix:
        text = clean_spaces(f"{text} {suffix}")
    return text

def iter_ingredients(recipe: dict) -> Iterable[dict]:
    """Iterate through all ingredient items in a recipe."""
    for block in recipe.get("ingredientBlocks", []):
        for it in block.get("ingredients", []):
            yield it
    # some recipes also have ingredients listed inside content sections
    for section in recipe.get("content", []):
        for it in section.get("ingredients", []) or []:
            yield it

def dedup_keep_order(items: List[str]) -> List[str]:
    """Remove duplicates but keep the original order."""
    seen = set()
    result = []
    for x in items:
        key = x.lower()
        if key not in seen:
            seen.add(key)
            result.append(x)
    return result


# --- load the JSON file ---
with open("einfachbacken_export_200_recipes_1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

recipes: List[Dict[str, Any]] = data.get("data", {}).get("recipeExport", [])


# --- single recipe ---
def ingredients_for_recipe(recipes: List[dict], recipe_name: str, dedup: bool = True) -> List[str]:
    recipe = next((r for r in recipes if r.get("name") == recipe_name), None)
    if not recipe:
        return []
    items = [join_ingredient_parts(it) for it in iter_ingredients(recipe)]
    return dedup_keep_order(items) if dedup else items
# --- all recipes combined ---
def ingredients_global(recipes: List[dict], dedup: bool = True) -> List[str]:
    items = []
    for r in recipes:
        items.extend(join_ingredient_parts(it) for it in iter_ingredients(r))
    return dedup_keep_order(items) if dedup else items


# --- per-recipe dictionary ---
def ingredients_per_recipe(recipes: List[dict], dedup: bool = True) -> Dict[str, List[str]]:
    result = {}
    for r in recipes:
        items = [join_ingredient_parts(it) for it in iter_ingredients(r)]
        result[r.get("name", "Unnamed")] = dedup_keep_order(items) if dedup else items
    return result


# --- example usage ---
one_recipe = ingredients_for_recipe(recipes, "Veganes Stollenkonfekt")
print("Single recipe:", one_recipe)

all_ingredients = ingredients_global(recipes)
print("Global:", all_ingredients[:20], f"... ({len(all_ingredients)} total)")

by_recipe = ingredients_per_recipe(recipes)
print("Recipes found:", list(by_recipe.keys())[:3])

Single recipe: ['Marzipan', 'Weizenmehl (Type 405)', 'Trockenhefe', 'Zucker', 'Vanillezucker', 'gemahlenen Mandeln', 'Stollengewürz', 'pflanzliche Milch', 'vegane Butter (Zimmertemperatur)', 'Rosinen', 'Orangeat', 'Zitronat', 'Saft (oder Rum)', 'Etwas Mehl zum Arbeiten', 'Puderzucker', 'Vanillinzucker', 'gemahlene Mandeln', 'veganeButter (Zimmertemperatur)', 'EtwasMehl zum Arbeiten']
Global: ['Marzipan', 'Weizenmehl (Type 405)', 'Trockenhefe', 'Zucker', 'Vanillezucker', 'gemahlenen Mandeln', 'Stollengewürz', 'pflanzliche Milch', 'vegane Butter (Zimmertemperatur)', 'Rosinen', 'Orangeat', 'Zitronat', 'Saft (oder Rum)', 'Etwas Mehl zum Arbeiten', 'Puderzucker', 'Vanillinzucker', 'gemahlene Mandeln', 'veganeButter (Zimmertemperatur)', 'EtwasMehl zum Arbeiten', 'Ei (Größe M)'] ... (743 total)
Recipes found: ['Veganes Stollenkonfekt', 'Wickeltorte', 'Orangen-Dessert mit Joghurt']


In [18]:
translated_texts_all = []

In [ ]:
for src_texts in all_ingredients:
    inputs = tokenizer(src_texts, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)

    # Decode and print result
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    # print(translated_texts[0])
    translated_texts_all.append(translated_texts[0])

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(".csv")
